In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Building a Decision Tree from Scratch [35 points]

### **Node Representation [3 points]**

- **Purpose**: Represents a node in the decision tree, which can either be a leaf or an internal node.
- **Attributes**:
  - `is_leaf` (bool): To determine if the node is a leaf node.
  - `feature_index` (int): The feature index used to split.
  - `threshold` (float): The threshold value used to split the data.
  - `prediction` (int): The value of the prediction (if the node is a leaf).
  - `left` (dict): Left child node.
  - `right` (dict): Right child node.

In [2]:
def create_node(
    is_leaf: bool,
    feature_index: int = -1,
    threshold: float = -1,
    prediction: float = None,
) -> dict:
    """
    Creates a decision tree node.

    Args:
        is_leaf (bool): Whether the node is a leaf node.
        feature_index (int): The feature index used for splitting.
        threshold (float): The threshold value for splitting.
        prediction (float): The prediction value for leaf nodes.

    Returns:
        dict: A dictionary representing a decision tree node.
    """
    # fill code here

    return {
        "is_leaf": is_leaf,
        "feature_index": feature_index,
        "threshold": threshold,
        "prediction": prediction,
        "left": None,
        "right": None
    }

### **Calculating Entropy [7 points]**

- **Purpose**: Calculate the entropy for a given split of labels.
-
  - Entropy is a measure of impurity, where lower values indicate a purer split.
  - For each subset of labels (left and right), calculate the proportion of each class.
  - Use the formula for entropy
  - The entropy for the split is the weighted average of the left and right entropies.


In [3]:
def calculate_entropy(left_labels: pd.Series, right_labels: pd.Series) -> float:
    """
    Calculates the entropy for a split.

    Args:
        left_labels (pd.Series): The label values for the left split.
        right_labels (pd.Series): The label values for the right split.

    Returns:
        float: The entropy value for the split.
    """
    left_size = len(left_labels)
    right_size = len(right_labels)
    total_size = left_size + right_size

    if total_size == 0:
        return 0

    def entropy(labels):

        # fill code here
        counts = labels.value_counts()
        probabilities = counts / len(labels)
        return -sum(prob * np.log2(prob) for prob in probabilities if prob > 0)


    # Calculate entropy for both left and right splits
    entropy_left = entropy(left_labels)
    entropy_right = entropy(right_labels)

    # Weighted average of both entropies
    # fill code here
    weighted_entropy = (left_size / total_size) * entropy_left + (right_size / total_size) * entropy_right

    return weighted_entropy

### **Splitting Dataset [4 points]**

- **Purpose**: Split the dataset into left and right parts based on a given feature and threshold.
-
  - Iterate through each instance in the dataset.
  - Compare the value of the selected feature with the given threshold.
  - Append the instance to either the left or right split based on whether the feature value is less than or equal to the threshold.
  - Make sure both the features and labels are split according

In [4]:
def split_dataset(
    df: pd.DataFrame, feature_index: str, threshold: float
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Splits the dataset based on a feature and a threshold.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        feature_index (str): The feature to split on.
        threshold (float): The threshold value to use for the split.

    Returns:
        tuple: A tuple containing the left and right datasets.
    """
    #fill code here
    left_df = df[df[feature_index] <= threshold]
    right_df = df[df[feature_index] > threshold]

    return left_df, right_df

### **Checking Purity [2 points]**

- **Purpose**: Determine if all the labels are the same.
-
  - If all labels are identical, the node should be considered pure, and it will become a leaf node.
  - Check if all elements are the same.
  - This function helps determine whether further splitting is necessary.

In [5]:
def is_pure(labels: pd.Series) -> bool:
    """
    Checks if all the labels are the same.

    Args:
        labels (pd.Series): The label values.

    Returns:
        bool: True if all labels are the same, False otherwise.
    """
    # fill code here
    return labels.nunique() == 1

### **Finding the Best Split [7 points]**

- **Purpose**: Find the best feature and threshold to split the dataset.
-
  - Iterate through each feature and determine possible thresholds.
  - Use entropy as the metric to evaluate the quality of the split.
  - For each threshold, calculate the entropy of the resulting splits and choose the one with the lowest entropy.
  - The best split will minimize the entropy, meaning the resulting splits are as pure as possible.


In [6]:
def find_best_split(
    df: pd.DataFrame, features: list[str], label_column: str
) -> tuple[str, float]:
    """
    Finds the best feature and threshold to split the dataset by minimizing entropy.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        features (list[str]): The list of feature column names.
        label_column (str): The label column name.

    Returns:
        tuple: The feature name and threshold value for the best split.
    """
    best_entropy = float("inf")
    best_feature = ""
    best_threshold = -1

    # fill code here
    for feature in features:
        values = df[feature].unique()
        for threshold in values:
            left_df, right_df = split_dataset(df, feature, threshold)
            left_labels = left_df[label_column]
            right_labels = right_df[label_column]

            entropy = calculate_entropy(left_labels, right_labels)

            if entropy < best_entropy:
                best_entropy = entropy
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold

### **Calculating Prediction [2 points]**

- **Purpose**: Calculate the prediction value for a leaf node (most common label).
- **Description**:
  - This function returns the most common label in the dataset, which is used as the prediction for a leaf node.


In [7]:
def calculate_prediction(labels: pd.Series) -> int:
    """
    Calculates the prediction value for a leaf node.

    Args:
        labels (pd.Series): The label values.

    Returns:
        int: The most common value of the labels.
    """
    # fill code here
    return labels.mode()[0]

### **Building the Decision Tree [7 points]**

- **Purpose**: Recursively build the decision tree by finding the best split and creating nodes.
- **Hints**:
  - Start with the current depth as 0 and increase it with each recursive call.
  - Stop the recursion if the current depth reaches the maximum depth or if the node is pure.
  - Use the `find_best_split` function to determine the best feature and threshold for splitting.
  - Create left and right child nodes recursively and attach them to the current node.

In [8]:
def build_tree(
    df: pd.DataFrame,
    features: list[str],
    label_column: str,
    current_depth: int,
    max_depth: int,
) -> dict:
    """
    Builds the decision tree recursively.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        features (list[str]): The list of feature column names.
        label_column (str): The label column name.
        current_depth (int): The current depth of the tree.
        max_depth (int): The maximum allowed depth of the tree.

    Returns:
        dict: The root node of the decision tree.
    """
    # fill code for base case
    if current_depth >= max_depth or is_pure(df[label_column]):
        prediction = calculate_prediction(df[label_column])
        return create_node(is_leaf=True, prediction=prediction)


    # fill code to find best feature
    best_feature, best_threshold = find_best_split(df, features, label_column)

    # fill code to split the data
    left_split, right_split = split_dataset(df, best_feature, best_threshold)

    # recursively building left and right subtrees
    left_node = build_tree(left_split, features, label_column, current_depth + 1, max_depth)
    right_node = build_tree(right_split, features, label_column, current_depth + 1, max_depth)

    # Create current internal node
    node = create_node(
        is_leaf=False,
        feature_index=best_feature,
        threshold=best_threshold
    )
    node["left"] = left_node
    node["right"] = right_node


    return node

### **Predicting Using the Tree [3 points]**

- **Purpose**: Predict the label for a given feature vector using the trained decision tree.
- **Hints**:
  - Start at the root of the tree and traverse down to a leaf node.
  - For each internal node, decide whether to move left or right based on the feature value and threshold.
  - When a leaf node is reached, return its prediction value.

In [9]:
def predict(tree: dict, sample: pd.Series) -> int:
    """
    Predicts the label for a given feature vector using the decision tree.

    Args:
        tree (dict): The root node of the decision tree.
        sample (pd.Series): The feature vector.

    Returns:
        int: The predicted label.
    """
    # fill code here
    node = tree

    # Traverse until reaching a leaf node
    while not node["is_leaf"]:
        feature_val = sample[node["feature_index"]]
        threshold = node["threshold"]

        if feature_val <= threshold:
            node = node["left"]
        else:
            node = node["right"]

    return node["prediction"]

### Using a Real-World Dataset and Comparison with Scikit-Learn

In [10]:
# Load the Iris dataset
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [11]:
# Prepare features and labels
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
label_column = 'class'
iris['class'] = iris['class'].apply(lambda x: 0 if x == 'Iris-setosa' else (1 if x == 'Iris-versicolor' else 2))

In [12]:
# Split the dataset into training and testing
X_train, X_test = train_test_split(iris, test_size=0.2, random_state=42)

In [13]:
max_depth = 3
decision_tree = build_tree(X_train, features, label_column, 0, max_depth)

In [14]:
predictions = X_test.apply(lambda row: predict(decision_tree, row), axis=1)

In [15]:
custom_accuracy = accuracy_score(X_test[label_column], predictions)
print(f"Custom Decision Tree Accuracy: {custom_accuracy:.2f}")

Custom Decision Tree Accuracy: 1.00


In [16]:
clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
clf.fit(X_train[features], X_train[label_column])
sklearn_predictions = clf.predict(X_test[features])
sklearn_accuracy = accuracy_score(X_test[label_column], sklearn_predictions)
print(f"Scikit-Learn Decision Tree Accuracy: {sklearn_accuracy:.2f}")

Scikit-Learn Decision Tree Accuracy: 1.00


# Random Forests (BONUS) [10 points]

Using the algorithm discussed in class, implement the random forest algorithm. You must reuse some of the Decision Trees functions you wrote above. You must call and use the random forest algorithm you build on the IRIS dataset.

In [17]:
def bootstrap_sample(df: pd.DataFrame) -> pd.DataFrame:
    # creating a bootstrap sample of input dataframe
    sample = df.sample(n=len(df), replace=True)
    return sample

In [18]:
import random

def build_random_forest(df: pd.DataFrame, features: list, label_column: str, n_trees: int, max_depth: int,
                  max_features_ratio: float = 0.5) -> dict:
    forest = {}

    for i in range(n_trees):
        bootstrapped_df = bootstrap_sample(df)

        # Randomly select a subset of features
        n_features = int(len(features) * max_features_ratio)
        sampled_features = random.sample(features, n_features)

        tree = build_tree(bootstrapped_df, sampled_features, label_column, 0, max_depth)
        forest[f"tree_{i}"] = (tree, sampled_features)

    return forest


In [19]:
def forest_prediction(forest: dict, sample: pd.Series) -> int:
    from collections import Counter
    predictions = []

    for tree_key in forest:
        tree, tree_features = forest[tree_key]
        prediction = predict(tree, sample[tree_features])
        predictions.append(prediction)

    return Counter(predictions).most_common(1)[0][0]


In [20]:
# Load the Iris dataset
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [21]:
# Prepare features and labels
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
label_column = 'class'
iris['class'] = iris['class'].apply(lambda x: 0 if x == 'Iris-setosa' else (1 if x == 'Iris-versicolor' else 2))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(iris[features], iris[label_column], test_size=0.2, random_state=42, shuffle=True)

In [23]:
# preparing training data
train_df = X_train.copy()
train_df[label_column] = y_train

# training forest
n_trees = 10
max_depth = 3
forest = build_random_forest(train_df, features, label_column, n_trees, max_depth)

# calculating accuracy
rf_predictions = 0
for i in range(len(X_test)):
    test_sample = X_test.iloc[i]
    prediction = forest_prediction(forest, test_sample)
    if prediction == y_test.iloc[i]:
        rf_predictions += 1

rf_accuracy = rf_predictions / len(X_test)
print(f"Accuracy: {rf_accuracy}")

Accuracy: 0.9666666666666667


# Logistic Regression [15 points]

### 1. Show that: [5 points]

$$
\sigma(-x) = 1 - \sigma(x)
$$

where
$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

**Answer:**

<br>

Given the definition of $\sigma(x)$, we know that:

$$\sigma(-x) = \frac{1}{1 + e^{-(-x)}} = \frac{1}{1 + e^x}$$

and

$$1 - \sigma(x) = 1 - \frac{1}{1 + e^{-x}} = \frac{(1 + e^{-x}) - 1}{1 + e^{-x}} = \frac{e^{-x}}{1 + e^{-x}} = \frac{1}{1 + e^x}$$

<br>

Since $\sigma(-x)$ and $ 1-\sigma(x) $ are both equal to $\frac{1}{1 + e^x}$, this proves that $\sigma(-x) = 1 - \sigma(x)$.




### 2. Show that: [5 points]

$$
\frac{d}{dx}\sigma(x) = \sigma(x)(1 - \sigma(x))
$$


**Answer:**

<br>

From part 1, we are given that $\sigma(x) = \frac{1}{1 + e^{-x}}$

<br>

Taking the derivative of $\sigma(x)$, we have:

$$\frac{d}{dx} \sigma(x) = \frac{d}{dx} \left( \frac{1}{1 + e^{-x}} \right)
= \frac{e^{-x}}{(1 + e^{-x})^2}$$

<br>

From part 1, we also know that $1 - \sigma(x) = \frac{e^{-x}}{1 + e^{-x}}$

<br>

If we multiply $\sigma(x)$ and $1-\sigma(x)$, we have:

$$
\sigma(x)(1 - \sigma(x)) = \left( \frac{1}{1 + e^{-x}} \right) \left( \frac{e^{-x}}{1 + e^{-x}} \right) = \frac{e^{-x}}{(1 + e^{-x})^2}
$$

<br>

Since the product of $\sigma(x)(1 - \sigma(x))$ is equivalent to the derivative of $\sigma(x)$, we have therefore shown that $\frac{d}{dx} \sigma(x) = \sigma(x)(1 - \sigma(x))$.

### 3. Gradient Descent [5 points]
You have a univariate function you wish to minimize, $f(w) = 5(w-11)^4$. Suppose you wish to perform gradient descent with constant step size $\alpha = 1/40$. Starting with $w_0 = 13$, perform 5 steps of gradient descent. What are $w_0, w_1, w_2, w_3, w_4, w_5$? What is the value of $f(w_5)$?

**Answer:**

<br>

Given $f(w) = 5(w - 11)^4$, we compute the gradient of $f(w)$ using the chain rule:

$$f'(w) = \frac{d}{dw} \left[ 5(w - 11)^4 \right] = 20(w - 11)^3$$

<br>

Therefore the gradient descent update rule is:

$$w_{t+1} = w_t - \alpha \cdot f'(w_t) = w_t - \frac{1}{40} \cdot 20(w_t - 11)^3 = w_t - \frac{1}{2}(w_t - 11)^3$$

<br>

Starting with $w_0 = 13$, we can compute the following five steps:

$$w_0 = 13$$

$$w_1 = 13 - \frac{1}{2}(13 - 11)^3 = 13 - \frac{1}{2}(2^3) = 13 - 4 = 9$$

$$w_2 = 9 - \frac{1}{2}(9 - 11)^3 = 9 - \frac{1}{2}(-2)^3 = 9 + 4 = 13$$

$$w_3 = 13 - \frac{1}{2}(13 - 11)^3 = 13 - 4 = 9$$

$$w_4 = 9 - \frac{1}{2}(9 - 11)^3 = 9 + 4 = 13$$

$$w_5 = 13 - \frac{1}{2}(13 - 11)^3 = 13 - 4 = 9$$

<br>

Therefore, the value of $f(w_5)$ is:

$$w_5 = 9$$  
$$f(w_5) = 5(9 - 11)^4 = 5(-2)^4 = 5 \cdot 16 = 80$$

<br>

Thus:
$$f(w_5) = 80$$
